In [1]:
import numpy as np
import os
import pandas as pd
import h5py
import seaborn as sns
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from toolbox.utils import set_random_seed
from toolbox.plt_utils import density_scatter

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%autoreload 2

In [3]:
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
# for TF To find gpu
os.environ["LD_LIBRARY_PATH"] = f"{os.environ['CONDA_PREFIX']}/lib:" + os.environ.get("LD_LIBRARY_PATH", "")

In [4]:
# set random seed for reproducibility
set_random_seed(seed=42)

2026-01-18 12:03:53.889305: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/valehvpa/miniconda3/envs/hist6/lib:/usr/local/cuda/lib64:/usr/local/cuda/lib64
2026-01-18 12:03:53.889350: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


TensorFlow not installed, skipping tf seeding.


/home/valehvpa/miniconda3/envs/hist6/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Seed set to 42


# Prep atac_hgp_map for hv2 to run on encode gm12878

I took this from 1_17_2025.ipynb.

Ran /home/valehvpa/projects/scCisTrans/histobpnet/experiments/valeh/scripts/narrowPeaks_to_summits_modified.sh to generate summits bed file for the correct peak set (peaks.all_input_regions.ENCSR868FGK.bed.gz).

Also ran this manually first as bedtools closest needs files to be lexicographically sorted according to their webapge and chatgpt.
Note here Im using the narrow peaks from encode directly, not peaks.bed from anusri, im not exactly sure what the difference is but hopefully it doesnt matter a lot for now.

In [ ]:
# zcat /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/encode/gm12878_h3k27ac/ENCFF023LTU.bed.gz | sort -k1,1 -k2,2n -c
# -> didnt return 0, so I re sorted using the below

In [ ]:
# gunzip -k /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/encode/gm12878_h3k27ac/ENCFF023LTU.bed.gz

# sort -k1,1 -k2,2n /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/encode/gm12878_h3k27ac/ENCFF023LTU.bed > \
#     /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/encode/gm12878_h3k27ac/ENCFF023LTU.sorted.lexicographic.bed

In [ ]:
# sort -k1,1 -k2,2n -c /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/encode/gm12878_h3k27ac/ENCFF023LTU.sorted.lexicographic.bed
# echo $?
# > ran this to make sure the first command returns nothing and the second returns 0 (meaning the file is sorted)

In [5]:
p = "/large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/encode/gm12878_h3k27ac/ENCFF023LTU.sorted.lexicographic.bed"

histone_df = pd.read_csv(
    p,
    sep="\t",
    header=None,
    index_col=False,
)
histone_df["length"] = histone_df[2] - histone_df[1]
histone_df.rename(columns={0: 'chrom', 1: 'chromStart', 2: 'chromEnd'}, inplace=True)

assert histone_df.duplicated().any() == False, "There are duplicate rows in the histone peaks file."
print(histone_df.shape)

(53689, 11)


In [6]:
def run_closest_for_window_reverse(window: int, base_dir_override: str, atac_bed = None):
    print("*** Running for window size: ", window)

    # read atac peaks, keep only chrom, chromStart, chromEnd
    atac_df = pl.read_csv(
        atac_bed,
        separator="\t",
        has_header=False,
        new_columns=[
            "chrom", "chromStart", "chromEnd", "name", "score", "strand",
        ],
    ).select(["chrom", "chromStart", "chromEnd"])

    # select cols
    pl.from_pandas(histone_df).select(["chrom", "chromStart", "chromEnd"]).write_csv(
        os.path.join(base_dir_override,"temp_B.bed"),
        separator="\t",
        include_header=False
    )
    atac_df.select(["chrom", "chromStart", "chromEnd"]).write_csv(
        os.path.join(base_dir_override,"temp_A.bed"),
        separator="\t",
        include_header=False
    )

    # https://bedtools.readthedocs.io/en/latest/content/tools/closest.html
    fn_prefix = f"reverse_narrow_peak_histone_closest_{str(window)}bp"
    # -t first returns the "first" closest feature in B in case of ties
    cmd = f"cd {base_dir_override} && \
        bedtools closest -a temp_A.bed -b temp_B.bed -t first > {fn_prefix}_atac.bed && \
        command rm -rf temp_A.bed && \
        command rm -rf temp_B.bed"
    os.system(cmd)

In [7]:
bdo = "/large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/gm12878_atac_encode/processed"
run_closest_for_window_reverse(
    2114,
    base_dir_override=bdo,
    atac_bed=os.path.join(bdo, "peaks.all_input_regions.ENCSR637XSC.summits.2114bp.sorted.lexicographic.bed.gz"),
)

*** Running for window size:  2114


In [8]:
!head /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/gm12878_atac_encode/processed/reverse_narrow_peak_histone_closest_2114bp_atac.bed

/bin/bash: /home/valehvpa/miniconda3/envs/hist6/lib/libtinfo.so.6: no version information available (required by /bin/bash)
chr1	8995	11109	chr1	778215	778711
chr1	777346	779460	chr1	778215	778711
chr1	777610	779724	chr1	778215	778711
chr1	777755	779869	chr1	778215	778711
chr1	778062	780176	chr1	778215	778711
chr1	803509	805623	chr1	826667	827622
chr1	803857	805971	chr1	826667	827622
chr1	804457	806571	chr1	826667	827622
chr1	825985	828099	chr1	826667	827622
chr1	826264	828378	chr1	826667	827622


The two counts below should be teh same (ie number of atac peaks is unchanged after bedtools closest)

In [9]:
!zcat /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/gm12878_atac_encode/processed/peaks.all_input_regions.ENCSR637XSC.summits.2114bp.sorted.lexicographic.bed.gz | wc -l

/bin/bash: /home/valehvpa/miniconda3/envs/hist6/lib/libtinfo.so.6: no version information available (required by /bin/bash)
277907


In [11]:
!wc -l /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/gm12878_atac_encode/processed/reverse_narrow_peak_histone_closest_2114bp_atac.bed

/bin/bash: /home/valehvpa/miniconda3/envs/hist6/lib/libtinfo.so.6: no version information available (required by /bin/bash)
277907 /large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/gm12878_atac_encode/processed/reverse_narrow_peak_histone_closest_2114bp_atac.bed


In [12]:
p = "/large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/gm12878_atac_encode/processed/reverse_narrow_peak_histone_closest_2114bp_atac.bed"
df = pd.read_csv(p, sep="\t", header=None, names=["chrom_atac", "start_atac", "end_atac", "chrom_histone", "start_histone", "end_histone"])
df

,chrom_atac,start_atac,end_atac,chrom_histone,start_histone,end_histone
0,chr1,8995,11109,chr1,778215,778711
1,chr1,777346,779460,chr1,778215,778711
2,chr1,777610,779724,chr1,778215,778711
3,chr1,777755,779869,chr1,778215,778711
4,chr1,778062,780176,chr1,778215,778711
...,...,...,...,...,...,...
277902,chrX,156012422,156014536,chrX,155997760,155998263
277903,chrX,156012694,156014808,chrX,155997760,155998263
277904,chrX,156015787,156017901,chrX,155997760,155998263
277905,chrY,4990899,4993013,.,-1,-1


In [13]:
df[df["chrom_histone"] == "."]

,chrom_atac,start_atac,end_atac,chrom_histone,start_histone,end_histone
277905,chrY,4990899,4993013,.,-1,-1
277906,chrY,15354512,15356626,.,-1,-1


## constrain to 1k distance

skip the atac peaks who are not within Xbp of the hgp (in the sense coded below). Adn for those I set hist_chrom to -1 so that in the code after setting skip_missing_hist these will be effectively skipped.

In [14]:
p = "/large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/gm12878_atac_encode/processed/reverse_narrow_peak_histone_closest_2114bp_atac.bed"
atac_hgp_df = pd.read_csv(p, sep="\t", header=None, names=[
    "chrom", "start", "end", "hist_chrom", "hist_start", "hist_end"
])

In [15]:
# if end is < hist_start - 2000, or start > hist_end + 2000, then set out_of_bounds to True
X = 2000
atac_hgp_df["out_of_bounds"] = (atac_hgp_df["end"] < atac_hgp_df["hist_start"] - X) | (atac_hgp_df["start"] > atac_hgp_df["hist_end"] + X)
# if hist_chrom is ".", then also set out_of_bounds to True
atac_hgp_df.loc[atac_hgp_df["hist_chrom"] == ".", "out_of_bounds"] = True
atac_hgp_df["out_of_bounds"].sum() * 100 / len(atac_hgp_df)

55.482949331970765

In [16]:
# if end is < hist_start - 1000, or start > hist_end + 1000, then set out_of_bounds to True
X = 1000
atac_hgp_df["out_of_bounds"] = (atac_hgp_df["end"] < atac_hgp_df["hist_start"] - X) | (atac_hgp_df["start"] > atac_hgp_df["hist_end"] + X)
# if hist_chrom is ".", then also set out_of_bounds to True
atac_hgp_df.loc[atac_hgp_df["hist_chrom"] == ".", "out_of_bounds"] = True
atac_hgp_df["out_of_bounds"].sum() * 100 / len(atac_hgp_df)

56.72581115265179

In [17]:
idx = atac_hgp_df["out_of_bounds"] == True
atac_hgp_df.loc[idx, "hist_chrom"] = "."
atac_hgp_df.loc[idx, "hist_start"] = "-1"
atac_hgp_df.loc[idx, "hist_end"] = "-1"
# set dtype to int
atac_hgp_df["hist_start"] = atac_hgp_df["hist_start"].astype(int)
atac_hgp_df["hist_end"] = atac_hgp_df["hist_end"].astype(int)
del atac_hgp_df["out_of_bounds"]

/tmp/ipykernel_1586487/316362100.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  atac_hgp_df.loc[idx, "hist_start"] = "-1"
/tmp/ipykernel_1586487/316362100.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-1' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  atac_hgp_df.loc[idx, "hist_end"] = "-1"


In [ ]:
p = "/large_storage/goodarzilab/valehvpa/data/projects/scCisTrans/for_hist/gm12878_atac_encode/processed/reverse_narrow_peak_histone_closest_2114bp_atac_within_1k_bound.bed"
atac_hgp_df.to_csv(
    p,
    sep="\t",
    header=False,
    index=False
)